In [0]:
%sql
CREATE OR REPLACE TEMP VIEW orders_items
AS
SELECT * FROM 
VALUES
(1, array('smartphone', 'laptop', 'monitor')),
(2, array('tablet', 'headphones', 'smartwatch')),
(3, array('keyboard', 'mouse'))
AS orders(order_id, items);


In [0]:
%sql
SELECT * FROM orders_items;

#####1. Convert all items name to uppercase using Transform Function

In [0]:
%sql
SELECT order_id,
      transform(items, x -> UPPER(x)) as upper_items
    FROM orders_items;

#####2. filter only item contains 'smart' using Filter Function

In [0]:
%sql
SELECT 
        order_id,
        FILTER(items, x -> x LIKE '%smart%') AS smart_items
FROM orders_items;

##### 3. Checking whether order_items includes any 'Monitor' using Exists Function

In [0]:
%sql
SELECT order_id,
      EXISTS(items, x -> x = 'monitor') AS has_monitor
FROM orders_items;

##### Array with more than one item

In [0]:
%sql
CREATE OR REPLACE TEMP VIEW order_items
AS
SELECT * FROM VALUES
(1, array(
          named_struct('name', 'smartphone', 'price', 699),
          named_struct('name', 'laptop', 'price', 1199),
          named_struct('name', 'monitor', 'price', 399)
          )),
(2, array(
          named_struct('name', 'tablet', 'price', 599),
          named_struct('name', 'headphones', 'price', 199),
          named_struct('name', 'smartwatch', 'price', 299)
          )),
(3, array(
          named_struct('name', 'keyboard', 'price', 89),
          named_struct('name', 'mouse', 'price', 59)
          ))
AS orders(order_id, items);


In [0]:
%sql
SELECT * FROM order_items;

##### 1. convert all item name to upper case and add 10% tax to all prices given

In [0]:
%sql
SELECT order_id,
      TRANSFORM(items, x -> named_struct(
                            'name', UPPER(x.name),
                            'price', ROUND(x.price * 1.10, 2)
      )) AS transformed_items
FROM order_items;

##### 2. calculate the total order amount for each of the orders (Aggregate functions)

In [0]:
%sql
SELECT order_id,
        aggregate(items, 0, (acc, x) -> acc + x.price) as total_order_price
    FROM order_items;

#### Map functions

##### Commonly used map functions

In [0]:
%sql
CREATE OR REPLACE TEMP VIEW order_item_prices
AS
SELECT * FROM VALUES
(1, MAP('smartphone', 699, 'laptop', 1199, 'monitor', 399)),
(2, MAP('tablet', 599, 'headphones', 199, 'smartwatch', 299)),
(3, MAP('keyboard', 89, 'mouse', 59))
AS orders(order_id, item_prices);

In [0]:
%sql
SELECT * FROM order_item_prices;

#####1. convert all items name to upper case

In [0]:
%sql
SELECT order_id,
      transform_keys(item_prices, (item, price) -> UPPER(item)) as Items
FROM order_item_prices;

#####2. filter only items with price more than 400

In [0]:
%sql
SELECT order_id,
      map_filter(item_prices, (item, price) -> price > 400) as items
  FROM order_item_prices;